In [ ]:
#import libraries
import pandas as pd
!pip install tmdbv3api
!pip install tmdbsimple
from tmdbv3api import TMDb
from tmdbv3api import Movie
from tmdbv3api import Person
import tmdbsimple as tmdbs
import seaborn as sns
api_key = '5bacb21ac761e60782a3ef72ade82c7f'
tmdbs.API_KEY = api_key

In [ ]:
# read and view the data
imdb_df = pd.read_csv("movies.csv", encoding = "ISO-8859-1")
print("The dataset shape:",imdb_df.shape)
print(imdb_df.describe().transpose())
imdb_df.head(5)

In [ ]:
# drop the movies when budget = 0
imdb_df = imdb_df[imdb_df.budget != 0]

In [ ]:
# drop the movies that aren't made in USA
imdb_df = imdb_df[imdb_df.country == "USA"]

In [ ]:
print("The dataset shape:",imdb_df.shape)
print(imdb_df.describe().transpose())
imdb_df.head()

In [ ]:
# check if there is any missing value
print('missing values in the data: \n', imdb_df.isnull().sum())

In [ ]:
# reset the index on the cleaned data
imdb_df.reset_index(inplace=True)

In [ ]:
# check if index changed
imdb_df.tail()

In [ ]:
# Call the tmdb api 
tmdb = TMDb()
tmdb.api_key = api_key
tmdb.language = 'en'
tmdb.debug = True

In [ ]:
# Get director popularity from API
director_popularity = dict()
for i in imdb_df.index:
    search = tmdbs.Search()
    response = search.person(query=imdb_df.director[i])
    try:
      for s in search.results:
        if s['known_for_department'] == 'Directing':
            # print(s['id'], s['popularity'], s['known_for_department'], s['profile_path'])
            director_popularity[i] = s['popularity']
            break
    except KeyError:
      continue
print(director_popularity)

In [ ]:
# Get actor popularity from API
actor_popularity = dict()
for i in imdb_df.index:
    search = tmdbs.Search()
    response = search.person(query=imdb_df.star[i])
    try:
      for s in search.results:
          if s['known_for_department'] == 'Acting':
              # print(s['id'], s['popularity'], s['known_for_department'], s['profile_path'])
              actor_popularity[i] = s['popularity']
              break
    except KeyError:
      continue
print(actor_popularity)

In [ ]:
# Get writer popularity from API
writer_popularity = dict()
for i in imdb_df.index:
    search = tmdbs.Search()
    response = search.person(query=imdb_df.writer[i])
    try:
      for s in search.results:
        if s['known_for_department'] == 'Writing':
          # print(s['id'], s['popularity'], s['known_for_department'], s['profile_path'])
          writer_popularity[i] = s['popularity']
          break      
    except KeyError:
      continue
        
print(writer_popularity)

In [ ]:
# create new dataframes to merge with dataset
df1  = pd.DataFrame(director_popularity, index=director_popularity.keys())
df1 = df1.transpose()
df1.drop(df1.iloc[:, 1:], axis = 1, inplace = True)
df1.rename(columns={1: "director_popularity"}, inplace=True)

df2  = pd.DataFrame(actor_popularity, index=actor_popularity.keys())
df2 = df2.transpose()
df2.drop(df2.iloc[:, 1:], axis = 1, inplace = True)
df2.rename(columns={0: "actor_popularity"}, inplace=True)


df3  = pd.DataFrame(writer_popularity, index=writer_popularity.keys())
df3 = df3.transpose()
df3.drop(df3.iloc[:, 1:], axis = 1, inplace = True)
df3.rename(columns={1: "writer_popularity"}, inplace=True)

In [ ]:
print(df1.head())
print(df2.head())
print(df3.head())

In [ ]:
# create the new dataset with the popularity values
new_df = imdb_df
new_df = pd.concat([imdb_df, df1], join='inner', ignore_index=False, axis=1)
new_df = pd.concat([new_df, df2], join='inner', ignore_index=False, axis=1)
new_df = pd.concat([new_df, df3], join='inner', ignore_index=False, axis=1)
new_df


In [ ]:
# check that they have been added
new_df.info()

In [ ]:
# export dataset to CSV file
new_df.to_csv('movie_dataset')